Code adapted from the official BERT notebook:
https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb

### Prepare Data for BERT

In [ ]:
import os
import datetime

import wget
import tqdm
import zipfile

import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical

In [ ]:
reuters = pd.read_pickle('reuters_news_concatenated.pkl', 'bz2').sample(frac=1).reset_index()
clusters = 4
labels = np.copy(reuters.Y)
labels[reuters.Y<np.percentile(reuters.Y, 100/clusters)] = 0
for i in range(1, clusters):
    labels[reuters.Y>np.percentile(reuters.Y, 100*i/clusters)] = i
reuters.Y = labels.astype("int")

In [ ]:
reuters_BERT = pd.DataFrame(columns=['label', 'text'])

In [ ]:
reuters_BERT.label = reuters.Y
reuters_BERT.text = reuters.news.apply(lambda x: ' '.join(x))

In [ ]:
reuters_BERT.head()

In [ ]:
mask = np.random.rand(len(reuters_BERT)) < 0.8

train = reuters_BERT[mask]
test = reuters_BERT[~mask]

### Download Pre-Trained Model

In [ ]:
wget.download('https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/modeling.py')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/optimization.py')
wget.download('https://raw.githubusercontent.com/google-research/bert/master/tokenization.py')

In [ ]:
wget.download('https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip')

In [ ]:
with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall()

### Fine-Tune BERT

In [ ]:
import modeling
import optimization
import run_classifier
import tokenization

import tensorflow as tf

In [ ]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = 'uncased_L-12_H-768_A-12'
OUTPUT_DIR = 'bert_output'

In [ ]:
def create_examples(lines, set_type, labels=None):
# Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 512
# Model configs
SAVE_CHECKPOINTS_STEPS = 10000 #if you wish to finetune a model on a larger dataset, use larger interval
ITERATIONS_PER_LOOP = 10000
NUM_TPU_CORES = 4
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(4)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(train.text.values, 'train', labels=train.label.values)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

In [ ]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

### Evaluate BERT predictions

In [ ]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
    """Creates an `input_fn` closure to be passed to TPUEstimator."""

    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    all_label_ids = []

    for feature in features:
        all_input_ids.append(feature.input_ids)
        all_input_mask.append(feature.input_mask)
        all_segment_ids.append(feature.segment_ids)
        all_label_ids.append(feature.label_id)

    def input_fn(params):
        """The actual input function."""
        print(params)
        batch_size = 500

        num_examples = len(features)

        d = tf.data.Dataset.from_tensor_slices({
            "input_ids":
                tf.constant(
                    all_input_ids, shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "input_mask":
                tf.constant(
                    all_input_mask,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "segment_ids":
                tf.constant(
                    all_segment_ids,
                    shape=[num_examples, seq_length],
                    dtype=tf.int32),
            "label_ids":
                tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
        })

        if is_training:
            d = d.repeat()
            d = d.shuffle(buffer_size=100)

        d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
        return d

    return input_fn

In [ ]:
predict_examples = create_examples(test.text, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

In [ ]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy of BERT is: ", accuracy_score(test.label,preds))
from sklearn.metrics import matthews_corrcoef
print("MCC of BERT is: ", matthews_corrcoef(test.label,preds))